In [4]:
from dotenv import load_dotenv
load_dotenv()

# 필요한 라이브러리 import

True

In [5]:
# 필요한 라이브러리 import
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.schema import Document


In [6]:
# 1. 문서 로드하기 - 로컬 document.txt 파일에서 텍스트 로드
document_path = "document.txt"

# TextLoader로 문서 로드
loader = TextLoader(document_path, encoding='utf-8')
documents = loader.load()

print(f"문서 로드 완료: {len(documents)}개 문서")
print(f"첫 번째 문서 길이: {len(documents[0].page_content)} 문자")
print(f"문서 내용 미리보기:\n{documents[0].page_content[:500]}...")


문서 로드 완료: 1개 문서
첫 번째 문서 길이: 70255 문자
문서 내용 미리보기:
Chapter 3

'There are three stages in your reintegration,' said O'Brien. 'There is
learning, there is understanding, and there is acceptance. It is time for
you to enter upon the second stage.'

As always, Winston was lying flat on his back. But of late his bonds were
looser. They still held him to the bed, but he could move his knees a
little and could turn his head from side to side and raise his arms from
the elbow. The dial, also, had grown to be less of a terror. He could
evade its pangs if...


In [7]:
# 2. 문서 쪼개기 - CharacterTextSplitter를 사용해서 문서를 작은 조각들로 나누기
text_splitter = CharacterTextSplitter(
    chunk_size=1000,  # 각 청크의 최대 문자 수
    chunk_overlap=200,  # 청크 간 겹치는 문자 수
    separator="\n"  # 문단 단위로 분할
)

# 문서를 청크로 분할
text_chunks = text_splitter.split_documents(documents)

print(f"문서 분할 완료: {len(text_chunks)}개 청크")
print(f"첫 번째 청크 길이: {len(text_chunks[0].page_content)} 문자")
print(f"첫 번째 청크 내용:\n{text_chunks[0].page_content[:300]}...")


문서 분할 완료: 87개 청크
첫 번째 청크 길이: 984 문자
첫 번째 청크 내용:
Chapter 3
'There are three stages in your reintegration,' said O'Brien. 'There is
learning, there is understanding, and there is acceptance. It is time for
you to enter upon the second stage.'
As always, Winston was lying flat on his back. But of late his bonds were
looser. They still held him to th...


In [8]:
# 3. 임베딩 생성 및 캐시 - OpenAIEmbeddings와 CacheBackedEmbeddings 사용
from langchain.storage import LocalFileStore

# OpenAI API 키 설정 (환경변수에서 가져오기)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # 필요시 주석 해제하고 API 키 입력

# 기본 임베딩 모델 생성
underlying_embeddings = OpenAIEmbeddings()

# 로컬 파일 시스템에 임베딩 캐시 저장
store = LocalFileStore("./cache/")

# 캐시된 임베딩 생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings, store, namespace=underlying_embeddings.model
)

print("임베딩 모델 및 캐시 설정 완료")


/tmp/ipykernel_175711/2869102504.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  underlying_embeddings = OpenAIEmbeddings()


임베딩 모델 및 캐시 설정 완료


In [9]:
# 4. 벡터 스토어 생성 - FAISS를 사용해서 임베딩된 문서들을 저장하고 검색 가능한 데이터베스 만들기
vectorstore = FAISS.from_documents(text_chunks, cached_embeddings)

# 벡터 스토어를 로컬에 저장 (선택사항)
vectorstore.save_local("faiss_index")

print(f"FAISS 벡터 스토어 생성 완료")
print(f"저장된 문서 수: {vectorstore.index.ntotal}")

# 검색 테스트
test_query = "Aaronson"
docs = vectorstore.similarity_search(test_query, k=3)
print(f"\n'{test_query}' 검색 결과:")
for i, doc in enumerate(docs):
    print(f"문서 {i+1}: {doc.page_content[:200]}...")


FAISS 벡터 스토어 생성 완료
저장된 문서 수: 87

'Aaronson' 검색 결과:
문서 1: of its own accord. He wrote:
GOD IS POWER
He accepted everything. The past was alterable. The past never had been
altered. Oceania was at war with Eastasia. Oceania had always been at war
with Eastasi...
문서 2: impossible even to rise from the horizontal if it had not been for the
bottle and teacup placed beside the bed overnight. Through the midday
hours he sat with glazed face, the bottle handy, listening ...
문서 3: of walking in sunlight, and an armed guard at his back. The long-hoped-for
bullet was entering his brain.
He gazed up at the enormous face. Forty years it had taken him to learn
what kind of smile was...


In [10]:
# 5. 대화 메모리와 질문 처리 - ConversationBufferMemory를 사용해 대화 기록 관리
# 6. 체인 연결 - 모든 컴포넌트들을 Stuff Documents 체인으로 연결

# 대화 메모리 생성
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# LLM 모델 생성
llm = OpenAI(temperature=0)

# ConversationalRetrievalChain 생성 (Stuff Documents 체인 사용)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory=memory,
    chain_type="stuff",  # Stuff Documents 체인 사용
    verbose=True
)

print("RAG 파이프라인 구성 완료!")
print("- LLM: OpenAI")
print("- 임베딩: OpenAI Embeddings (캐시됨)")
print("- 벡터 스토어: FAISS")
print("- 메모리: ConversationBufferMemory")
print("- 체인 타입: Stuff Documents")


RAG 파이프라인 구성 완료!
- LLM: OpenAI
- 임베딩: OpenAI Embeddings (캐시됨)
- 벡터 스토어: FAISS
- 메모리: ConversationBufferMemory
- 체인 타입: Stuff Documents


/tmp/ipykernel_175711/319349485.py:5: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
/tmp/ipykernel_175711/319349485.py:11: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [11]:
# 7. 질문 테스트 - 주어진 3개 질문으로 RAG 파이프라인 테스트

# 질문 리스트
questions = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?"
]

print("=" * 60)
print("RAG 파이프라인 질문 테스트")
print("=" * 60)

for i, question in enumerate(questions, 1):
    print(f"\n질문 {i}: {question}")
    print("-" * 40)
    
    try:
        # 질문에 대한 답변 생성
        response = qa_chain({"question": question})
        answer = response["answer"]
        
        print(f"답변: {answer}")
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
    
    print("-" * 40)

print("\n" + "=" * 60)
print("모든 질문 테스트 완료!")
print("=" * 60)


RAG 파이프라인 질문 테스트

질문 1: Aaronson 은 유죄인가요?
----------------------------------------


/tmp/ipykernel_175711/1901385183.py:20: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain({"question": question})




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

of its own accord. He wrote:
GOD IS POWER
He accepted everything. The past was alterable. The past never had been
altered. Oceania was at war with Eastasia. Oceania had always been at war
with Eastasia. Jones, Aaronson, and Rutherford were guilty of the crimes
they were charged with. He had never seen the photograph that disproved
their guilt. It had never existed, he had invented it. He remembered
remembering contrary things, but those were false memories, products of
self-deception. How easy it all was! Only surrender, and everything else
followed. It was like swimming against a current that swept you backwards
however hard you struggled, and then suddenly deciding to turn round and
go with the current instead of oppos

In [12]:
print("\n" + "=" * 60)
print("RAG 파이프라인 구현 완료!")
print("=" * 60)
print("구현된 컴포넌트:")
print("1. ✅ 문서 로드: TextLoader로 로컬 document.txt 파일에서 텍스트 로드")
print("2. ✅ 문서 분할: CharacterTextSplitter로 청크 생성")
print("3. ✅ 임베딩 캐시: OpenAIEmbeddings + CacheBackedEmbeddings")
print("4. ✅ 벡터 스토어: FAISS로 검색 가능한 데이터베이스 구축")
print("5. ✅ 대화 메모리: ConversationBufferMemory로 대화 기록 관리")
print("6. ✅ 체인 연결: ConversationalRetrievalChain (Stuff Documents)")
print("7. ✅ 질문 테스트: 3개 질문으로 파이프라인 검증")
print("=" * 60)



RAG 파이프라인 구현 완료!
구현된 컴포넌트:
1. ✅ 문서 로드: TextLoader로 로컬 document.txt 파일에서 텍스트 로드
2. ✅ 문서 분할: CharacterTextSplitter로 청크 생성
3. ✅ 임베딩 캐시: OpenAIEmbeddings + CacheBackedEmbeddings
4. ✅ 벡터 스토어: FAISS로 검색 가능한 데이터베이스 구축
5. ✅ 대화 메모리: ConversationBufferMemory로 대화 기록 관리
6. ✅ 체인 연결: ConversationalRetrievalChain (Stuff Documents)
7. ✅ 질문 테스트: 3개 질문으로 파이프라인 검증
